In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

import rnn
import train

In [2]:
embedding_size = 10
hidden_size = 35
output_size = len(rnn.char.keys())
n_layers = 2
batch_size = 16
GAMMA = 0.99 # for exponential moving avarage constant from paper




In [3]:
print(rnn.token_num)

9


In [4]:
model = rnn.BFgen(rnn.token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [5]:
print(train.objective_PG(model, lambda x : np.array([1] * model.batch_size), N=16), model.entropy)

(Variable containing:
-2.0816
[torch.FloatTensor of size 1]
, Variable containing:
 3323.5195
[torch.FloatTensor of size 1]
)


rnn.py:110: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = self.logsoftmax(decoded.permute(2, 0, 1)) # output_size x length x batch


In [6]:
print(train.objective_PQT(model))

Variable containing:
-208.3249
[torch.FloatTensor of size 1]



In [7]:
print(model.pqt_programs)

['-<>-.>+<[,+[,,<[[,[-..[<]<>.]-+]><,+>+]>+,.[-..[.>+[<->+-[-,[,+<<<+,.+.<++[.+.<+-<>]<,[<<,[,,>.]><.>'
 '>>+].<,,.>--[.]<<-<-+,+].[<,>+--.][]+]<><+<]]>.<<,+<<<],-]<<--]+>.,-]]<>[+>.+.+>]],+-[.>+,<.>,]>->+,'
 '+<>--.>+<+.,>+<+.++[-<<[.],-+,-++.-.]<][],<<],<-.<.][.+<-][[+>-]<,,[][.[[,.<],[<,<,<,.]>[<>>,.>[+][<'
 '][].>++-+-[-,,<.-.,<.>.+->,+[][[[<<<>+-]+,+]--.[.[+[>-,<>-,>[,.>>>,.>+-<<<>--[-,.<<+[,><]]<[-+[[+>[<'
 '[]<[]+++>-+,.[[].,,,+-+.]]--]-,<-.+++<,<+,+<+<<]<+[[]-[][<>-,.[+[]-<,-.>-<-]--..[+,[<,.-]]-[,,],<,+['
 '.>-]]+-[>>..<.-]+..>[-.<,+]>+<+.[]++,[<->.+.>>-.[->]+-<[+>-][>+[.>]+-.>++,+--.->.[><-][.,>>]-[-+.][.'
 '[++>>,>-]-[<+-..<<>.>[<<,-.-<,-.,]+.<,[-<><,<+>,]+<>-.>[+[<><>>],>[.>.-]-[>+<<[<+<>,->-[-]<>]<+-]-,-'
 '][<],[.,>,[<<>+<+<].-<<+>[.[][+<+]<<-,.+],>.+[.>-]>[[[.<-.+<<]-.>]><]+,+-[>-+]>>->[<<,].]<..,[],<-++'
 '+[>-,][<<,.,[-.<,.-.+<>>,..-]><,,]]>-][,[],+-[.][,[>>[>,]>]--+.][,<>,][[-[-><]].-++[+[<+]]<-++<>->..'
 ']-+>.+<],++,[[]>,<<<,.[..+>-><<.-[><+]-[>>[,+.<]]<]],.>>+-<+-.

In [8]:
bfAdd = ",>++++++[<-------->-],[<+>-]<++++++++."
bfAdd2 = ",>++++++[<-------->-],[<+>-]<."
bfAdd3 = ",>++++++[<-------->-],[<+>-]<++..."
bfAdd4 = ",>++++++[<-------->-],[<+>-]<+++............"
addBatch = [bfAdd, bfAdd2, bfAdd3, bfAdd4]
bfAddIn = ['11', '12', '23']
bfAddOut = ['2','3','5']

In [9]:
reward = train.batch_reward(bfAddIn, bfAddOut)

In [10]:
reward(addBatch)

array([ 0.99609375,  1.        , -1.        , -1.        ])

In [11]:
reverse_in = ["a", "ab", "abc", "abcd", "abcde"]
reverse_out = ["a", "ba", "cba", "dcba", "edcba"]
reverse_reward = train.batch_reward(reverse_in, reverse_out)

In [12]:
train.train_pqt_pg(model, reverse_reward)

Epoch Obj  Sample
----- ----- ------


KeyboardInterrupt: 

In [13]:
expected_code = "><.><"
program_batch_code = [".....", "..<..", "[][][][][[][[][][][][][][]]]", "]"]

In [14]:
simple_rew = train.simplified_batch_reward(expected_code)

In [15]:
simple_rew(program_batch_code)

array([ 0.996875  ,  0.99609375, -1.        ,  0.19921875])

In [16]:
training_expected_code = ">>>.." * 20
print len(training_expected_code),training_expected_code

100 >>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..


In [17]:
training_rew = train.simplified_batch_reward(training_expected_code)
training_rew(program_batch_code)

array([0.04988281, 0.04988281, 0.27890625, 0.00996094])

In [ ]:
train.train_pqt_pg(model, training_rew)

Epoch Obj  Sample
----- ----- ------
Variable containing:
1.00000e-06 *
 -4.2764
[torch.FloatTensor of size 1]

Variable containing:
-207.9940
[torch.FloatTensor of size 1]

Variable containing:
1.00000e+05 *
  2.0938
[torch.FloatTensor of size 1]

0.9965281579639621
   0  -8305.910 ]>[]]<,<,.+<>.->.,[,+<<]-->].[]+<+]<+-,>-.>.<--.+>..[><-.++>,[+><-]]-[]<]],].<<[[>]<.-,<[.<,..,++-<[
Variable containing:
1.00000e-06 *
  1.0715
[torch.FloatTensor of size 1]

Variable containing:
-208.0531
[torch.FloatTensor of size 1]

Variable containing:
1.00000e+05 *
  2.0936
[torch.FloatTensor of size 1]

0.996557867835036
   1  -8309.009 [.,>+>,],[<-[<>].,[.]>,->[<[[[.<-[.+]<+.<-,]-.-<+]]->.]]]>]-,]]+<,,.>.-++,.-.<[>]]].]>+,+[,.]<,<[,<.
Variable containing:
1.00000e-06 *
 -5.3834
[torch.FloatTensor of size 1]

Variable containing:
-208.0918
[torch.FloatTensor of size 1]

Variable containing:
1.00000e+05 *
  2.0934
[torch.FloatTensor of size 1]

0.9965867695457521
   2  -8311.223 >-],><+[,]<.<-+<[..><